In [1]:
from google.colab import drive
drive.mount('/content/drive/')
DIR='/content/drive/MyDrive/Research/OntoConnectWithGNN/GNN_1/'

import os
os.chdir(DIR)

Mounted at /content/drive/


In [2]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'''))

!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

!pip install git+https://github.com/facebookresearch/fastText.git
!pip install git+https://github.com/facebookresearch/PyTorch-BigGraph.git

<IPython.core.display.Javascript object>

1.7.0+cu101
10.1
  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-h18m_f3j
  Running command git clone -q https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-h18m_f3j
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3088322 sha256=4fdcbbda266dd42c3b3855e1e8c5a82f138d62be9cd4924b4b9475e5915ea0f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-32_k067h/wheels/69/f8/19/7f0ab407c078795bc9f86e1f6381349254f86fd7d229902355
Successfully built fasttext
  Cloning https://github.com/facebookresearch/PyTorch-BigGraph.git to /tmp/pip-req-build-45m7pact
  Running command git clone -q https://github.com/facebookresearch/PyTorch-BigGraph.git /tmp/pip-req-build-45m7pact
  Created wheel for torchbiggraph: filename=torchbiggraph-1.0.1.dev0-cp37-none-any.whl size=118953 sha256=67f0e492abc6351b532d674ae06362d0808c89988a5e184f481f1ef4b5bebd64
  Stored in directory: /tmp/pip-ephem-wheel-cache-4tzavbo6/wheels/d5/d0/e8/9fa5

In [3]:
#from OntoSimImports import *
%run OntoSimImports.py

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
#import OntoSimConstants
%run OntoSimConstants.py
from OntoSimConstants import *

In [5]:
def assignVar():
    conf = {
        'dict_fl': DATA_DIR+'dict/dict.txt',
        'dict_json': DATA_DIR+'dict/dict_fast.json',
        'model': 'fast.bin',
        'vec_fl': vec_nm
    }

    return conf

In [6]:
def readDict(conf):
    with open(code_path + conf["dict_fl"], 'r') as f:
        entity_dict = f.readlines()

    return entity_dict

In [7]:
def loadDictVec(entity_dict, fasttext_model):
    fast_dict = {}
    for entity_word in entity_dict:
        entity_word = entity_word.replace("\n", "")
        vec = fasttext_model.get_word_vector(entity_word)
        fast_dict[entity_word] = vec.tolist()

    return fast_dict

In [8]:
def writeDictVec(conf, entity_dict):
    with open(code_path + conf['dict_json'], 'w') as outfile:
        json.dump(entity_dict, outfile, indent=4)

In [9]:
def loadPreTrainedDictVec(conf):
    fl_nm = faxt_text_model_path + conf['vec_fl']
    with open(fl_nm) as f:
        data = json.load(f)
    return data

In [10]:
def chkAllKeyPresentInVec(entity_dict_pretrained, entity_dict_current):
    for entity_key in entity_dict_current:
        entity_key = entity_key.replace("\n", "")
        if(entity_key not in entity_dict_pretrained.keys()):
            return False
    return True

In [11]:
def dictToVecFile(ind):
    print("Creating Vector from "+ ind)
    conf = assignVar()
    entity_dict_pretrained = loadPreTrainedDictVec(conf)
    entity_dict_current = readDict(conf)
    if(chkAllKeyPresentInVec(entity_dict_pretrained, entity_dict_current)):
        writeDictVec(conf, entity_dict_pretrained)
    else:
        raise Exception("Sorry, key is missing in pre-trained vector file")

In [12]:
def dictToVecModel(ind):
    conf = assignVar()
    print("Creating Vector from " + ind + " " + conf['model'])
    print('fastText model Loads START:- ' + str(datetime.datetime.now()))
    fasttext_model = load_model(faxt_text_model_path + conf["model"])
    print("Model Dimension:- " + str(fasttext_model.get_dimension()))
    print('fastText model Loads END:- ' + str(datetime.datetime.now()))

    entity_dict = readDict(conf)
    entity_dict_vec = loadDictVec(entity_dict, fasttext_model)
    writeDictVec(conf, entity_dict_vec)

In [13]:
def dictToVec(ind):
    try:
        print("#################### DictionaryToVector START ####################")
        if(ind == frm_vic_1):
            dictToVecModel(frm_vic_1)
        elif(ind == frm_vic_2):
            dictToVecFile(frm_vic_2)

        time.sleep(wait_time)
    except Exception as exp:
        raise exp
    finally:
        print("#################### DictionaryToVector FINISH ####################")

In [14]:
### MAIN FUNCTION 
if __name__=="__main__":
  dictToVec(frm_vic_1)

#################### DictionaryToVector START ####################
Creating Vector from model fast.bin
fastText model Loads START:- 2021-02-24 19:59:47.608912
Model Dimension:- 300
fastText model Loads END:- 2021-02-24 20:02:03.843461
#################### DictionaryToVector FINISH ####################
